# YOLO code

In [ ]:
!pip install ultralytics

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolo11n.pt")  # load a pretrained model (recommended for training)
# model = YOLO("/kaggle/input/coco-nn/yolo11n_coco128.pt")

In [ ]:
# Train the model on modified dataset
results = model.train(data="/kaggle/input/cocomini/coco_minitrain/coco_minitrain.yaml", epochs=50,imgsz=640)

In [ ]:
# predict on random image
model.predict("/kaggle/input/coco-2017-dataset/coco2017/train2017/000000000247.jpg", save=True, imgsz=320, conf=0.5)

Show image

In [ ]:
import cv2
from matplotlib import pyplot as plt

# Load the image
image = cv2.imread('/kaggle/working/runs/detect/train2/000000000247.jpg')

# Convert to RGB
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Display the image
plt.imshow(image)
plt.axis('off')  # Hide axes
plt.show()

In [5]:
model.save("/kaggle/working/model_name.pt")

Download csv for original COCO

In [ ]:
import json
import csv
a=[]
# Load the COCO data from the JSON file
with open("/kaggle/input/coco-2017-dataset/coco2017/annotations/instances_train2017.json", "r") as json_file:
    coco_data = json.load(json_file)

# Create a CSV file for writing
csv_file = open("coco_data.csv", "w", newline="")
csv_writer = csv.writer(csv_file)

# Write the header row to the CSV file
csv_writer.writerow(["image", "x1", "y1", "x2", "y2", "category_id","category"])

# Create a dictionary to map image IDs to file names
image_id_to_filename = {image["id"]: image["file_name"] for image in coco_data["images"]}

# Iterate through annotations and extract data
for annotation in coco_data["annotations"]:
    image_id = annotation["image_id"]
    image_name = image_id_to_filename.get(image_id, "N/A")
    # Use "N/A" if image ID is not found
    if(image_name=="N/A"):
        continue
    x1, y1, width, height = annotation["bbox"]
    x2 =  width
    y2 =  height
    category_id = annotation["category_id"]
    # Find the category name based on the category_id
    category = next((cat["name"] for cat in coco_data["categories"] if cat["id"] == category_id),"N/A")
    if(category=="N/A"):
        continue
    a.append(category)

    # Write the data to the CSV file
    csv_writer.writerow([image_name, x1, y1, x2, y2, category_id-1,category])

# Close the CSV file
csv_file.close()

print("COCO data has been converted to CSV and saved to coco_data.csv")

# Convert csv to txt

In [2]:
import csv
import os
from PIL import Image

# Define the directory containing the images
image_directory = '/kaggle/input/coco25k/images'
output_directory = '/kaggle/working/labels'
os.makedirs(output_directory, exist_ok=True)

# Class mapping
class_mapping = {
    "person": 0,
    "bicycle": 1,
    "car": 2,
    "motorcycle": 3,
    "airplane": 4,
    "bus": 5,
    "train": 6,
    "truck": 7,
    "boat": 8,
    "traffic light": 9,
    "fire hydrant": 10,
    "stop sign": 11,
    "parking meter": 12,
    "bench": 13,
    "bird": 14,
    "cat": 15,
    "dog": 16,
    "horse": 17,
    "sheep": 18,
    "cow": 19,
    "elephant": 20,
    "bear": 21,
    "zebra": 22,
    "giraffe": 23,
    "backpack": 24,
    "umbrella": 25,
    "handbag": 26,
    "tie": 27,
    "suitcase": 28,
    "frisbee": 29,
    "skis": 30,
    "snowboard": 31,
    "sports ball": 32,
    "kite": 33,
    "baseball bat": 34,
    "baseball glove": 35,
    "skateboard": 36,
    "surfboard": 37,
    "tennis racket": 38,
    "bottle": 39,
    "wine glass": 40,
    "cup": 41,
    "fork": 42,
    "knife": 43,
    "spoon": 44,
    "bowl": 45,
    "banana": 46,
    "apple": 47,
    "sandwich": 48,
    "orange": 49,
    "broccoli": 50,
    "carrot": 51,
    "hot dog": 52,
    "pizza": 53,
    "donut": 54,
    "cake": 55,
    "chair": 56,
    "couch": 57,
    "potted plant": 58,
    "bed": 59,
    "dining table": 60,
    "toilet": 61,
    "tv": 62,
    "laptop": 63,
    "mouse": 64,
    "remote": 65,
    "keyboard": 66,
    "cell phone": 67,
    "microwave": 68,
    "oven": 69,
    "toaster": 70,
    "sink": 71,
    "refrigerator": 72,
    "book": 73,
    "clock": 74,
    "vase": 75,
    "scissors": 76,
    "teddy bear": 77,
    "hair drier": 78,
    "toothbrush": 79,
    "background": 80
}

def normalize_values(x_center, y_center, width, height, image_width, image_height):
    x_center_normalized = (x_center + width / 2) / image_width
    y_center_normalized = (y_center + height / 2) / image_height
    width_normalized = width / image_width
    height_normalized = height / image_height
    return x_center_normalized, y_center_normalized, width_normalized, height_normalized

# Create a dictionary to hold lines for each image
output = {}

# Read from CSV file
with open('/kaggle/input/coco-minitrain/coco_minitrain2017.csv', mode='r') as file:
    reader = csv.reader(file, delimiter=',')  # Adjust delimiter if needed
    for row in reader:
#         print(row)
        image_name, x_center, y_center, width, height, class_name, _ = row
        class_id = class_mapping.get(class_name, 0)  # Default to 0 if not found

        # Convert values to float
        x_center = float(x_center)
        y_center = float(y_center)
        width = float(width)
        height = float(height)

        # Get the full path of the image
        image_path = os.path.join(image_directory, image_name)

        # Open the image to get dimensions
        with Image.open(image_path) as img:
            image_width, image_height = img.size

        # Normalize the values
        x_center_normalized, y_center_normalized, width_normalized, height_normalized = normalize_values(
            x_center, y_center, width, height, image_width, image_height
        )

        # Prepare the output line
        output_line = f"{class_id} {x_center_normalized:.6f} {y_center_normalized:.6f} {width_normalized:.6f} {height_normalized:.6f}"

        # Append to the corresponding image file
        if image_name not in output:
            output[image_name] = []
        output[image_name].append(output_line)

# Write the output to files
for image_name, lines in output.items():
    txt_filename = os.path.splitext(image_name)[0] + ".txt"
    output_path = os.path.join(output_directory, txt_filename)
    with open(output_path, 'w') as f:
        for line in lines:
            f.write(line + '\n')

In [ ]:
import shutil

shutil.make_archive('/kaggle/working/coco_label', 'zip', '/kaggle/working/labels')

In [ ]:
from IPython.display import FileLink

# Create a downloadable link for the zip file
FileLink('/kaggle/working/coco_label.zip')